In [1]:
from pathlib import Path  #paths in the object-oriented filesystem
import pandas as pd       #Python Data Analysis Library
import wget               #a program for downloading content from web servers
import time               #Module for time access and conversions 

from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import h5py

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt # plotting tool
import cartopy.feature as cfeature # to add coastlines, land and ocean
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib
matplotlib.use('agg')

import os
import pickle
import array as arr
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import tensorflow as tf

from tensorflow import keras
from sklearn.model_selection import ParameterGrid

from tensorflow.keras.layers import Input, Convolution2D, Convolution1D, MaxPooling2D, Dense, Dropout, \
                          Flatten, concatenate, Activation, Reshape, \
                          UpSampling2D,ZeroPadding2D, Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D

from tensorflow import keras
from sklearn.model_selection import ParameterGrid

from tensorflow.keras import Sequential
from tensorflow.keras import layers
from pylab import plt
import tensorflow as tf

from dask.diagnostics import ProgressBar
ProgressBar().register()

from keras.applications import Xception
from keras import layers
from keras import backend as K
from keras import optimizers
import keras

import matplotlib
matplotlib.use('agg')

import os
import pickle
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
from keras import layers
from keras.layers import Input, Convolution2D, Convolution1D, MaxPooling2D, Dense, Dropout, \
                          Flatten, concatenate, Activation, Reshape, \
                          UpSampling2D,ZeroPadding2D

import keras
from pylab import plt
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, figure
from dask.diagnostics import ProgressBar
ProgressBar().register()
import pickle
import json
import numpy as np
from tsmoothie.smoother import *
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [3]:
#Saved path
outdir = 'C:/Users/user/Research/Research Code/CIMP6/CMIP6_model/Ensemble_CMIP6/'
ifile = 'C:/Users/user/Research/Research Code/CIMP6/data/ensemble_train_overall_CMIP6/'
testsizepath='C:/Users/user/Research/Research Code/Compare_CNN_LSTM/Evaluation/CNN/y_testsize_359_32_64_1.nc'
y_test_size= xr.open_dataarray(testsizepath)

In [4]:
#Import data
ds = xr.open_dataset(ifile+'ensemble_CMIP6.nc', chunks={'time':1},decode_times=False)
ds

<xarray.Dataset>
Dimensions:    (time: 1380, bnds: 2, lon: 128, lat: 64)
Coordinates:
  * time       (time) float64 15.5 45.0 74.5 ... 4.193e+04 4.196e+04 4.199e+04
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lat        (lat) float64 87.86 85.1 82.31 79.53 ... -82.31 -85.1 -87.86
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) float64 dask.array<chunksize=(1, 2), meta=np.ndarray>
    tos        (time, lat, lon) float32 dask.array<chunksize=(1, 64, 128), meta=np.ndarray>
Attributes: (12/49)
    CDI:                    Climate Data Interface version 2.0.4 (https://mpi...
    source:                 ACCESS-CM2 (2019): \naerosol: UKCA-GLOMAP-mode\na...
    institution:            CSIRO (Commonwealth Scientific and Industrial Res...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variant_label:          r1i1p1f1
    version:                v20191108
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/0bcaaa74-aedb-4d45-a5e5-cb3ab467f2b5
    license:                CMIP6 model data produced by CSIRO is licensed un...
    CDO:                    Climate Data Operators version 2.0.4 (https://mpi...

In [5]:
#Add lev dimension
lev_coords = np.zeros(1)
ds_add_lev = ds.expand_dims({"lev": lev_coords})

In [6]:
ds_add_lev

<xarray.Dataset>
Dimensions:    (lev: 1, time: 1380, bnds: 2, lon: 128, lat: 64)
Coordinates:
  * lev        (lev) float64 0.0
  * time       (time) float64 15.5 45.0 74.5 ... 4.193e+04 4.196e+04 4.199e+04
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lat        (lat) float64 87.86 85.1 82.31 79.53 ... -82.31 -85.1 -87.86
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (lev, time, bnds) float64 dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    tos        (lev, time, lat, lon) float32 dask.array<chunksize=(1, 1, 64, 128), meta=np.ndarray>
Attributes: (12/49)
    CDI:                    Climate Data Interface version 2.0.4 (https://mpi...
    source:                 ACCESS-CM2 (2019): \naerosol: UKCA-GLOMAP-mode\na...
    institution:            CSIRO (Commonwealth Scientific and Industrial Res...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variant_label:          r1i1p1f1
    version:                v20191108
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/0bcaaa74-aedb-4d45-a5e5-cb3ab467f2b5
    license:                CMIP6 model data produced by CSIRO is licensed un...
    CDO:                    Climate Data Operators version 2.0.4 (https://mpi...

In [7]:
##Transpose dimensions before feeding them into the model
x = ds_add_lev.transpose('time','lat','lon','lev','bnds')
x

<xarray.Dataset>
Dimensions:    (lev: 1, time: 1380, bnds: 2, lon: 128, lat: 64)
Coordinates:
  * lev        (lev) float64 0.0
  * time       (time) float64 15.5 45.0 74.5 ... 4.193e+04 4.196e+04 4.199e+04
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lat        (lat) float64 87.86 85.1 82.31 79.53 ... -82.31 -85.1 -87.86
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, lev, bnds) float64 dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    tos        (time, lat, lon, lev) float32 dask.array<chunksize=(1, 64, 128, 1), meta=np.ndarray>
Attributes: (12/49)
    CDI:                    Climate Data Interface version 2.0.4 (https://mpi...
    source:                 ACCESS-CM2 (2019): \naerosol: UKCA-GLOMAP-mode\na...
    institution:            CSIRO (Commonwealth Scientific and Industrial Res...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variant_label:          r1i1p1f1
    version:                v20191108
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/0bcaaa74-aedb-4d45-a5e5-cb3ab467f2b5
    license:                CMIP6 model data produced by CSIRO is licensed un...
    CDO:                    Climate Data Operators version 2.0.4 (https://mpi...

In [8]:
#Sliced the area
data=x
data = data.sel(lat=slice(87,-2,1),lon=slice(100, 280,1))
data

<xarray.Dataset>
Dimensions:    (lev: 1, time: 1380, bnds: 2, lon: 64, lat: 32)
Coordinates:
  * lev        (lev) float64 0.0
  * time       (time) float64 15.5 45.0 74.5 ... 4.193e+04 4.196e+04 4.199e+04
  * lon        (lon) float64 101.2 104.1 106.9 109.7 ... 270.0 272.8 275.6 278.4
  * lat        (lat) float64 85.1 82.31 79.53 76.74 ... 6.977 4.186 1.395 -1.395
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, lev, bnds) float64 dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    tos        (time, lat, lon, lev) float32 dask.array<chunksize=(1, 32, 64, 1), meta=np.ndarray>
Attributes: (12/49)
    CDI:                    Climate Data Interface version 2.0.4 (https://mpi...
    source:                 ACCESS-CM2 (2019): \naerosol: UKCA-GLOMAP-mode\na...
    institution:            CSIRO (Commonwealth Scientific and Industrial Res...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variant_label:          r1i1p1f1
    version:                v20191108
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/0bcaaa74-aedb-4d45-a5e5-cb3ab467f2b5
    license:                CMIP6 model data produced by CSIRO is licensed un...
    CDO:                    Climate Data Operators version 2.0.4 (https://mpi...

In [9]:
#Normalized the data
mm = data.mean(('time','lat','lon'))
std = data.std(('time','lat','lon'))

mm.load()
std.load()

normalized = (data - mm) / std

normalized.to_netcdf(outdir + 'normalized_Ensemble_CMIP6.nc')

[########################################] | 100% Completed |  6.0s
[########################################] | 100% Completed |  6.0s
[########################################] | 100% Completed |  6.7s
[########################################] | 100% Completed |  6.8s
[########################################] | 100% Completed |  8.1s
[########################################] | 100% Completed |  8.1s


# Start tuning 

In [10]:
#Imported data
ds = xr.open_dataset(outdir + 'normalized_Ensemble_CMIP6.nc')

In [11]:
ds

<xarray.Dataset>
Dimensions:    (lev: 1, time: 1380, lon: 64, lat: 32, bnds: 2)
Coordinates:
  * lev        (lev) float64 0.0
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 1964-12-16T12:00:00
  * lon        (lon) float64 101.2 104.1 106.9 109.7 ... 270.0 272.8 275.6 278.4
  * lat        (lat) float64 85.1 82.31 79.53 76.74 ... 6.977 4.186 1.395 -1.395
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, lev, bnds) datetime64[ns] ...
    tos        (time, lat, lon, lev) float32 ...

In [12]:
#Prepared data
ds=ds['tos'].data #extra data array
ds[ds<0]=0 #treat nan values as 0
np.nan_to_num(ds,copy=False)
ds=ds/(np.nanmax(ds)) 
dates=pd.date_range(start='1855-01-01',periods=len(ds))
label=np.array(dates.month)
label=label-1

In [13]:
total_N = len(ds)
total_N

1380

In [14]:
#Splited the data

N_tune = 900
N_gab =120
N_dev = 360
N_test = 0
N_train = 0
N_spinup = total_N-(N_tune+N_gab+N_dev+N_test+N_train)

lead_time=1
x = ds

x = x.astype('float32')
x = x[:N_spinup+N_train+N_tune+N_gab+N_dev+N_test]

lat,lon,lev=x.shape[1:4]

In [15]:
def prepare_data(lead_time):
    ''' Shift the data based on the lead time and 
    divide it into a predictor set and a predictant set. 
    Then divide it into a tuning set, training set, development set, and a test set.'''
    if lead_time == 0:
        X = x
        y = X[:]
    else:

        X = x[:-lead_time]
        y = x[lead_time:]

    X_train = X[N_spinup+N_tune+N_dev+N_test:]
    y_train = y[N_spinup+N_tune+N_dev+N_test:]
    
    X_tune = X[N_spinup:N_spinup+N_tune]
    y_tune = y[N_spinup:N_spinup+N_tune]
    
    X_dev = X[N_spinup+N_tune+N_gab:N_spinup+N_tune+N_gab+N_dev]
    y_dev = y[N_spinup+N_tune+N_gab:N_spinup+N_tune+N_gab+N_dev]
    
    X_test = X[N_spinup+N_tune+N_dev:N_spinup+N_tune+N_dev+N_test]
    y_test = y[N_spinup+N_tune+N_dev:N_spinup+N_tune+N_dev+N_test]
    
       
    return X_tune,y_tune, X_dev, y_dev

In [16]:
# fixed (not-tuned params)
batch_size = 32
num_epochs = 5
pool_size = 2
drop_prob=0
conv_activation='relu'

param_string = '_'.join([str(e) for e in (N_train,lead_time,batch_size,num_epochs,pool_size,drop_prob)])

N_gpu = 2

In [17]:
def acc_score(x,y):
    '''timestepwise anomaly correlation coefficient, averaged over time
        (simple version without seasonal climatoloty)'''
    assert(x.shape==y.shape)
    return np.mean([np.corrcoef(x[i].flatten(),y[i].flatten())[0,1] for i in range(len(x))])

In [18]:
def build_model(conv_depth, kernel_size, hidden_size, n_hidden_layers, lr):

    model = keras.Sequential([
            
            ## Convolution which involves dimensionality reduction (similar to Encoder in an autoencoder)
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation, input_shape=(lat,lon,lev)),
            layers.MaxPooling2D(pool_size=pool_size),
            Dropout(drop_prob),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.MaxPooling2D(pool_size=pool_size),
            # end "encoder"
            
            
            # dense layers (Automatic flattening and reshaping occurs.)
            ] + [layers.Dense(hidden_size, activation='sigmoid') for i in range(n_hidden_layers)] +
             
            [
            
            
            # start "Decoder" (upsampling of the encoder above)
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.UpSampling2D(size=pool_size),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.UpSampling2D(size=pool_size),
            layers.Convolution2D(lev, kernel_size, padding='same', activation=None)
            ]
            )
    
    
    optimizer= keras.optimizers.Adam(lr=lr)


            
             
    #model.compile(loss='mean_squared_error', optimizer = optimizer)
    model.compile(loss=tf.keras.losses.binary_crossentropy,
     optimizer=tf.keras.optimizers.Adam(lr=0.001),
     #)
     metrics=[])
    
    
    return model


In [19]:
##See the data
X_tune,y_tune, X_dev, y_dev = prepare_data(lead_time)
print(X_tune.shape,X_dev.shape);
print(y_tune.shape,y_dev.shape)

(900, 32, 64, 1) (359, 32, 64, 1)
(900, 32, 64, 1) (359, 32, 64, 1)


In [20]:
##Hyperparameters
tunable_params = dict(
                  lr=[0.00001,0.00003,0.0001],
                  n_hidden_layers = [0,1,2],
                  kernel_size = [2,4,6],
                  hidden_size=[50,100,300,500],
                  conv_depth = [16,32]
        )

    
print(lead_time)

1


In [21]:
# All possible parameter configurations are iterated over. Scikit can be used for this.

param_grid = list(ParameterGrid(tunable_params))
n_param_combis = len(param_grid)
print('trying ',n_param_combis,' combinations')

res = []
for i,params in enumerate(param_grid):
    print('training on param set ',i)
    print(params)
    
   
    model = build_model(**params)
    
    print(model.summary())
    

    print('start training')
    hist = model.fit(X_tune, y_tune,
                       batch_size = batch_size,
             verbose=1, 
             epochs = num_epochs,
             validation_data=(X_dev,y_dev),
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                        min_delta=0,
                                        patience=5, # To ensure a great deal of patience before ending
                                        verbose=0, mode='auto'),
                       keras.callbacks.ModelCheckpoint('best_weights.h5', monitor='val_loss', 
                                                    verbose=1, save_best_only=True, 
                                                    save_weights_only=True, mode='auto', period=1)]
             )
    
    print('finished training')
    
    # To ensure the best model performance from the training based on learning curve,
    # Because the early stopping callback saves the model's "patience" epochs after the best one, this is required.
    model.load_weights('best_weights.h5')
    
    # delete the file generated by Model Checkppoint
    os.system('rm best_weights.h5')
    

    model.save_weights(outdir+'weights_'+param_string+'_tuning_paramset'+str(i)+'.h5')
    
    
    # reformat history
    
    hist =  hist.history

    y_dev_predicted = model.predict(X_dev)
    
    # compute accuracy
    rmse = np.sqrt(np.mean((y_dev_predicted - y_dev)**2))
    acc = acc_score(y_dev_predicted, y_dev)

    res.append(dict(hist=hist,params=params, scores=[rmse,acc]))
    
    plt.figure()
    plt.plot(hist['val_loss'], label='val_loss')
    plt.plot(hist['loss'], label='train loss')
    
    plt.legend()
    plt.savefig(outdir+'cnn_history'+param_string+'_tuning_paramset'+str(i)+'.png')
    
    pd.DataFrame(hist).to_csv(outdir+'history_'+param_string+'_tuning_paramset'+str(i)+'.csv')
    


pickle.dump(res,open(outdir+'tuning_result'+param_string+'.pkl','wb'))


C:\Users\nutta\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


trying  216  combinations
training on param set  0
{'conv_depth': 16, 'hidden_size': 50, 'kernel_size': 2, 'lr': 1e-05, 'n_hidden_layers': 0}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 64, 16)        80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 32, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 32, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 32, 16)        1040      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 16, 16)        0         
 2D)                                          

Epoch 1/5
27/29 [==========================>...] - ETA: 0s - loss: 1.0211
Epoch 1: val_loss improved from inf to 0.56446, saving model to best_weights.h5
29/29 [==============================] - 2s 44ms/step - loss: 1.0038 - val_loss: 0.5645
Epoch 2/5
27/29 [==========================>...] - ETA: 0s - loss: 0.4820
Epoch 2: val_loss improved from 0.56446 to 0.39754, saving model to best_weights.h5
29/29 [==============================] - 1s 35ms/step - loss: 0.4786 - val_loss: 0.3975
Epoch 3/5
27/29 [==========================>...] - ETA: 0s - loss: 0.3615
Epoch 3: val_loss improved from 0.39754 to 0.32085, saving model to best_weights.h5
29/29 [==============================] - 1s 39ms/step - loss: 0.3601 - val_loss: 0.3208
Epoch 4/5
29/29 [==============================] - ETA: 0s - loss: 0.3062
Epoch 4: val_loss improved from 0.32085 to 0.29334, saving model to best_weights.h5
29/29 [==============================] - 1s 42ms/step - loss: 0.3062 - val_loss: 0.2933
Epoch 5/5
27/29 [===

training on param set  1
{'conv_depth': 16, 'hidden_size': 50, 'kernel_size': 2, 'lr': 1e-05, 'n_hidden_layers': 1}
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 32, 64, 16)        80        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 32, 16)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 16, 32, 16)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 16, 32, 16)        1040      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 8, 16, 16)        0         
 2D)                                                             
    

Epoch 1/5
27/29 [==========================>...] - ETA: 0s - loss: 2.3624
Epoch 1: val_loss improved from inf to 1.51749, saving model to best_weights.h5
29/29 [==============================] - 2s 41ms/step - loss: 2.3274 - val_loss: 1.5175
Epoch 2/5
27/29 [==========================>...] - ETA: 0s - loss: 0.9365
Epoch 2: val_loss improved from 1.51749 to 0.62897, saving model to best_weights.h5
29/29 [==============================] - 1s 40ms/step - loss: 0.9241 - val_loss: 0.6290
Epoch 3/5
27/29 [==========================>...] - ETA: 0s - loss: 0.5899
Epoch 3: val_loss improved from 0.62897 to 0.55047, saving model to best_weights.h5
29/29 [==============================] - 1s 37ms/step - loss: 0.5883 - val_loss: 0.5505
Epoch 4/5
27/29 [==========================>...] - ETA: 0s - loss: 0.5072
Epoch 4: val_loss improved from 0.55047 to 0.46702, saving model to best_weights.h5
29/29 [==============================] - 1s 37ms/step - loss: 0.5054 - val_loss: 0.4670
Epoch 5/5
27/29 [===

...

Epoch 1/5
28/29 [===========================>..] - ETA: 0s - loss: 0.4996
Epoch 1: val_loss improved from inf to 0.28421, saving model to best_weights.h5
29/29 [==============================] - 10s 318ms/step - loss: 0.4986 - val_loss: 0.2842
Epoch 2/5
28/29 [===========================>..] - ETA: 0s - loss: 0.2785
Epoch 2: val_loss improved from 0.28421 to 0.26477, saving model to best_weights.h5
29/29 [==============================] - 10s 342ms/step - loss: 0.2785 - val_loss: 0.2648
Epoch 3/5
28/29 [===========================>..] - ETA: 0s - loss: 0.2642
Epoch 3: val_loss improved from 0.26477 to 0.25688, saving model to best_weights.h5
29/29 [==============================] - 10s 334ms/step - loss: 0.2642 - val_loss: 0.2569
Epoch 4/5
28/29 [===========================>..] - ETA: 0s - loss: 0.2579
Epoch 4: val_loss improved from 0.25688 to 0.25205, saving model to best_weights.h5
29/29 [==============================] - 9s 303ms/step - loss: 0.2579 - val_loss: 0.2521
Epoch 5/5
29/

training on param set  214
{'conv_depth': 32, 'hidden_size': 500, 'kernel_size': 6, 'lr': 0.0001, 'n_hidden_layers': 1}
Model: "sequential_214"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1070 (Conv2D)        (None, 32, 64, 32)        1184      
                                                                 
 max_pooling2d_428 (MaxPooli  (None, 16, 32, 32)       0         
 ng2D)                                                           
                                                                 
 dropout_214 (Dropout)       (None, 16, 32, 32)        0         
                                                                 
 conv2d_1071 (Conv2D)        (None, 16, 32, 32)        36896     
                                                                 
 max_pooling2d_429 (MaxPooli  (None, 8, 16, 32)        0         
 ng2D)                                                          

Epoch 1/5
29/29 [==============================] - ETA: 0s - loss: 4.2599
Epoch 1: val_loss improved from inf to 4.30918, saving model to best_weights.h5
29/29 [==============================] - 20s 657ms/step - loss: 4.2599 - val_loss: 4.3092
Epoch 2/5
29/29 [==============================] - ETA: 0s - loss: 4.2054
Epoch 2: val_loss did not improve from 4.30918
29/29 [==============================] - 19s 660ms/step - loss: 4.2054 - val_loss: 4.3092
Epoch 3/5
29/29 [==============================] - ETA: 0s - loss: 4.2054
Epoch 3: val_loss did not improve from 4.30918
29/29 [==============================] - 20s 693ms/step - loss: 4.2054 - val_loss: 4.3092
Epoch 4/5
29/29 [==============================] - ETA: 0s - loss: 4.2054
Epoch 4: val_loss did not improve from 4.30918
29/29 [==============================] - 18s 638ms/step - loss: 4.2054 - val_loss: 4.3092
Epoch 5/5
29/29 [==============================] - ETA: 0s - loss: 4.2054
Epoch 5: val_loss did not improve from 4.30918
29

training on param set  215
{'conv_depth': 32, 'hidden_size': 500, 'kernel_size': 6, 'lr': 0.0001, 'n_hidden_layers': 2}
Model: "sequential_215"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1075 (Conv2D)        (None, 32, 64, 32)        1184      
                                                                 
 max_pooling2d_430 (MaxPooli  (None, 16, 32, 32)       0         
 ng2D)                                                           
                                                                 
 dropout_215 (Dropout)       (None, 16, 32, 32)        0         
                                                                 
 conv2d_1076 (Conv2D)        (None, 16, 32, 32)        36896     
                                                                 
 max_pooling2d_431 (MaxPooli  (None, 8, 16, 32)        0         
 ng2D)                                                          

Epoch 1/5
29/29 [==============================] - ETA: 0s - loss: 7.6481
Epoch 1: val_loss improved from inf to 4.25510, saving model to best_weights.h5
29/29 [==============================] - 23s 762ms/step - loss: 7.6481 - val_loss: 4.2551
Epoch 2/5
29/29 [==============================] - ETA: 0s - loss: 3.4452
Epoch 2: val_loss did not improve from 4.25510
29/29 [==============================] - 20s 701ms/step - loss: 3.4452 - val_loss: 6.8578
Epoch 3/5
29/29 [==============================] - ETA: 0s - loss: 3.2394
Epoch 3: val_loss improved from 4.25510 to 2.67437, saving model to best_weights.h5
29/29 [==============================] - 21s 740ms/step - loss: 3.2394 - val_loss: 2.6744
Epoch 4/5
29/29 [==============================] - ETA: 0s - loss: 2.3878
Epoch 4: val_loss improved from 2.67437 to 2.21683, saving model to best_weights.h5
29/29 [==============================] - 21s 713ms/step - loss: 2.3878 - val_loss: 2.2168
Epoch 5/5
29/29 [==============================] 

In [23]:
##Read the accuracy
import pickle
import json

import numpy as np

res = pickle.load(open(outdir+'tuning_result0_1_32_5_2_0.pkl','rb'))

# get each run's final validating loss

final_val_losses = [e['hist']['val_loss'][-1] for e in res]

# convert to array
final_val_losses = np.array(final_val_losses)

# find the smallest index.
idx_smallest = np.argmin(final_val_losses)
res_best = res[idx_smallest]
print(res_best)

with open('tuning_best_result.txt','w') as f :
    f.write(str(res_best))

{'hist': {'loss': [0.47095590829849243, 0.2765541970729828, 0.26354387402534485, 0.25893470644950867, 0.25303205847740173], 'val_loss': [0.28442299365997314, 0.26581400632858276, 0.2591252326965332, 0.2510022222995758, 0.2478446066379547]}, 'params': {'conv_depth': 32, 'hidden_size': 100, 'kernel_size': 6, 'lr': 3e-05, 'n_hidden_layers': 0}, 'scores': [0.19067526, 0.9176578572244036]}


# END